In [4]:
from flask import Flask, render_template
from flask import request # to have access to the query string

In [2]:
app = Flask(__name__)

@app.route('/multiply')
@app.route('/multiply/<int:num1>/<int:num2>')
def multiply(num1=5, num2=5):
    res = 5*5
    return '{}'.format(res)


In [3]:
# templates in Flask
# long way

app = Flask(__name__)

@app.route('/multiply')
@app.route('/multiply/<int:num1>/<int:num2>')
def multiply(num1=5, num2=5):
# triple quotes keeps line breaks
    return """
    <!doctype html>
    <html>
    <head><title>Multiply!</title></head>
    <body>
    <h1>{} * {} = {}</h1>
    </body>
    </html>
    """.format(num1, num2, num1*num2)

In [ ]:
# short way - need to make a directory 
# - named templates - Flask will automatically look for it
# make name.html
# print variables - jinja2 - {{ num1 }}

@app.route('/multiply')
@app.route('/multiply/<int:num1>/<int:num2>')
def multiply(num1=5, num2=5):
    # need to pass the variables to render_template
    return render_template("name.html", num1=num1, num2=num2)

# or ...

def multiple(num1, num2):
    context = {'num1':num1, 'num2':num2}
    return render_template("name.html", **context)



In [ ]:
# template inheritance
''' put all the things that stay the same in layout.html
and make them into blocks {% block title%}{% endblock %}
{%   %} = command (as opposed to print or evaluate in jinja2)

when you use the inherited template in other .html files - 
add at the top:
{% extends "layout.html" %}
{% block title %}Howdy!{% endblock %} <-- overwrites the inherited title

if you want to add to a block & keep the inherited:
{% block title %}Howdy! | {{ super() }}{% endblock %}
{% block content %}

'''

In [ ]:
# static directory
''' this is where css goes
reference to this goes into the layout.html that is inherited by other 
name.html docs
<link rel="stylesheet" href="/static/styles.css"> <-- goes in the <head>

'''

In [ ]:
# javascript in static directory
'''
alert("Howdy");
and then need to add to layout.html:
goes in <body>:
{% block scripts %}{% endblock %}

goes where you need it to happen:
<script src="/static/scripts.js"></script> <-- or at least it did in this 
case bc that is where he wanted the thing to happen

'''

In [ ]:
from flask import redirect
from flask import url_for
'''
return redirect(url_for('index'))

'''


In [ ]:
# cookies
''' need json - import json
set cookies - set on response - the thing that goes back to the server 

import make_response from flask

-- need to do this bc you don't actually have the thing yet to send back 
-- so you have to make it!
def save():
    response = make_response(redirect(url_for('index')))
    response.set_cookie('character', json.dumps(dict(request.form.items())))
    return response


get the cookie back out:

def get_saved_data():
    try:
        data = json.loads(request.cookies.get('character'))
    except TypeError:
        data = {}
    return data

'''

In [ ]:
# looping through a list given to a template (in html)
<ul>                                     
{% for option in options %}
  <li>{{ option.name }}</li>
{% endfor %} 
</ul>

# option["name"] also works

In [ ]:
# showing selections
{% if saves.get('footwear') %}
<div class="footwear"><img src="/static/img/bear-{{ saves["footwear"] }}.svg"></div>
{% endif %}


In [ ]:
# flash messages
# in app.py, import flash from flask
# using session - and in flask those are cryptographically signed
# need to make secret key - just type random things - long is good

app = Flask(__name__)
app.secret_key = 'asdkjboeh238094tn;sdcnoad.k'

# & then update save method
# call flask method
@app.route('/save', methods=["POST"])
def save():
    flash("Alright!")
    ...

# but then need something that prints out the flash message
# layout.html - right before block content
<body>
    <div class="wrap no-bottom messages">
        {% with messages = get_flashed_messages() %}
        {% if messages %}
        <ul class="flashes">
            {% for message in messages %}
            <li> {{ message }}</li>
            {% endfor %}
        </ul>
        {% endif %}
        {% endwith %}
    </div>
    {% block content %}{% endblock %}

In [ ]:
# works with Heroku well

In [ ]:
# peewee - Python database

from peewee import *

from models import Challenge

all_challenges = Challenge.select()

Challenge.create(language= "Ruby", name= "Booleans")

sorted_challenges = Challenge.select().order_by(Challenge.steps)

In [ ]:
#!/usr/bin/env python3
import datetime
from collections import OrderedDict
import sys
import os

from peewee import *

db=SqliteDatabase('diary.db')


class Entry(Model):
    # content
    content = TextField()
    # timestamp - if you put parenthesis on .now below -it would be 
    # dt now, when you wrote it, not when you run it
    timestamp = DateTimeField(default=datetime.datetime.now)
    
    class Meta:
        database = db

def initialize():
    """Create the db and table if they don't exist"""
        db.connect()
        db.create_tables([Entry], safe=True)
        
def clear():
    # windows newer than 95 is int
    os.system('cls' if os.name =='nt' else 'clear')
        
def menu_loop():
    ''' show the menu'''
    choice = None
    
    while choice != 'q':
        clear()
        print("Enter 'q' to quit.")
        for key, value in menu.items():
            print('{}) {}'.format(key, value.__doc__))
        choice = input('Action:  ').lower().strip()
        
        if choice in menu:
            clear()
            menu[choice]()

def add_entry():
    """ Add an entry """
    print("Enter your entry, press ctrl+d when finished.")
    data = sys.stdin.read().strip()
    
    if data:
        if input('Save entry [Yn] ').lower() != 'n':
            Entry.create(content=data)
            print("Saved successfully!")
    
def view_entry(search_query=None):
    """ View all entries """
    entries = Entry.select().order_by(Entry.timestamp.desc())
    if search_query:
        entries = entries.where(Entry.content.contains(search_query))
    
    for entry in entries:
        timestamp = entry.timestamp.strftime('%A %B %d, %Y %I:%M%p')
        clear()
        print(timestamp)
        print('='*len(timestamp))
        print(entry.content)
        print('\n\n'+'='*len(timestamp))
        print('n) next entry')
        print('d) delete entry')
        print('q) return to main menu')
        
        next_action = input('Action: [Ndq]  ').lower().strip()
        if next_action =='q':
            break
        elif next_action == 'd':
            delete_entry(entry)

def search_entries():
    """ Search entries for a string."""
    view_entries(input('Search query: '))
    
def delete_entry(entry):
    """ Delete an entry. """
    if input("Are you sure? [yN]  ").lower() == "y":
        entry.delete_instance()
        print("Entry deleted!")
    
menu = OrderedDict([
    ('a', add_entry),
    ('v', view_entries),
    ('s', search_entries)
])

    
# To prevent code from being executed when you import a module, 
# put it into a function or class, or inside which if condition:
    
if __name__ == '__main__':
    initialize()
    menu_loop()

In [ ]:
# where with multiple conditions:
challenges = Challenge.select().where((Challenge.name == name) | (Challenge.language == language))


In [ ]:
# call os level stuff - import os
# os.system() lets us use system-level applications
def clear():
    # windows newer than 95 is int
    os.system('cls' if os.name =='nt' else 'clear')

In [ ]:
bio = TextField(default="") <-- empty string for its default makes it optional

In [ ]:
# !/usr/bin/env python3 <-- shebang line needed on linus/ workspaces/ mac
# if then run in command line: chmod +x diary.py, can run w/ ./diary.py

In [ ]:
# ordered dict as menu - remembers order you add it in, uses list


In [ ]:
# __doc__ --> gets docstring from function! cool

In [ ]:
# organize imports - stuff that comes from python, blank line, 3rd party stuff

In [5]:
!pip install flask_login

In [14]:
import datetime

from flask_login import UserMixin
from flask_bcrypt import generate_password_hash, check_password_hash
from peewee import *

db = SqliteDatabase('social.db')

class User(UserMixin, Model):
    username = CharField(unique=True)
    email = CharField(unique=True)
    password = CharField(max_length=100)
    joined_at = DateTimeField(default=datetime.datetime.now)
    is_admin = BooleanField(default=False)
    
    class Meta:
        database = db
        order_by = ('-joined_at',)

    # cls instead of self - bc you don't want to create an instance to 
    # call create_user to create an instance, so instead decorator & cls
    # so instead - this cls & decorator it will create the instance w/ cls
    # class methods are a great way to create classes from nothing
    @classmethod
    def create_user(cls, username, email, password, admin=False)
        try:
            cls.create(
                username=username,
                email=email,
                password=generate_password_hash(password),
                is_admin=admin)
        except IntegrityError:
            raise ValueError("User already exists")


In [15]:
# cryptographic hashing with Flask-Bcrypt
help(check_password_hash)

Help on function check_password_hash in module flask_bcrypt:

check_password_hash(pw_hash, password)
    This helper function wraps the eponymous method of :class:`Bcrypt.` It 
    is intended to be used as a helper function at the expense of the 
    configuration variable provided when passing back the app object. In other 
    words this shortcut does not make use of the app object at all.
    
    To this this function, simple import it from the module and use it in a 
    similar fashion as the method would be used. Here is a quick example::
        
        from flask.ext.bcrypt import check_password_hash
        check_password_hash(pw_hash, 'hunter2') # returns True
    
    :param pw_hash: The hash to be compared against.
    :param password: The password to compare.



In [ ]:
# beginning of every password $2a$ or $2y$ 
# next comes number of rounds, and then another $ and then a hash
# default is 12 rounds
# you could write it so the more times a password is failed, the more 
# rounds - it will take longer - eventually will not be worth the time
# the salt (extra bit of random data)
# to check - need another function - check_password_hash

In [ ]:
# social network with Flask

''' @classmethods - so you don't have to create an instance in order 
to create an instance

- need @classmethod
- need cls
'''

@classmethod
def create_user(cls, username, email):
    pass

In [ ]:
# app.py


from flask import Flask, g 
from flask_login import LoginManager

"""g = makes things global variables"""


import models

DEBUG = True
PORT = 8000
HOST = '0.0.0.0'

#could put in app.run - this is cleaner - change them all in one spot

app = Flask(__name__)
app.secret_key = 'asdkjb98q234r5_balunbsd#adjhk34' #needed for sessions

login_manager = LoginManager()
login_manager.init_app(app)
# if they are not loged in and they need to be - direct to 'login' view
login_manager.login_view = 'login'

@login_manager.user_loader
def load_user(userid):
    try:
        return models.User.get(models.User.id == userid)
    except models.DoesNotExist: #from peewee
        return None

@app.before_request
def before_request():
    """connect to the db before each request"""
    g.db = models.DATABASE
    g.db.connect()
    
@app.after_request
def after_request(response):
    """close the db connection after each request"""
    g.db.close()
    return response






if __name__ == '__main__':
    models.initialize()
    try:
        models.User.create_user(
            name='SarahBurgart',
            email='sburgart@gmail.com',
            password='password',
            admin=True)
    except ValueError:
        pass
    app.run(debug=DEBUG, host=HOST, port=PORT)


In [13]:
#models.py

@classmethod
def create_user


def initiailze():
    DATABASE.connect()
    DATABASE.create_tables([User], safe=True)
    DATABASE.close()

In [14]:
# forms- are about validation - default library for flask is wtf
# also supplies us with cross site forgery protection - csfp

# new file - forms.py
import flask_wtf import Form
from wtforms import StringField, PasswordField
from wtforms.validators import (DataRequired, Regexp, ValidationError, Email
                               Length, Equalto)
from models import User

def name_exists(form, field):
    if User.select().where(User.username == field.data).exists():
        raise ValidationError('User with that name already exists.')
        
def email_exists(form, field):
    if User.select().where(User.email == field.data).exists():
        raise ValidationError('User with that email already exists.')

class RegisterForm(Form):
    #label
    username = StringField(
        'Username',
        validators=[
            DataRequired(),
            Regexp(
            r'^[a-zA-Z0-9_]+$', #used ascii bc browsers sometimes don't play nicely with unicode
            message = ("Username should be one word, letters,"
                      "numbers, and underscores only.") # neat trick for two lines as one
            ),
            name_exists
        ])
    email = StringField(
        'Email',
        validators=[
            DataRequired(),
            Email(),
            email_exists     
        ])
    password = PasswordField(
        'Password',
        validators=[
            DataRequired(),
            Length(min=7),
            EqualTo('password2', message='Passwords much match')
        ])
    password2 = PasswordField(
        'Confirm Password',
        validators=[DataRequired()]
    )



In [ ]:
# we want to be able to render the form
from flask import (Flask, g, render_template, flash, redirect, url_for)
from flask_login import (LoginManager, login_user, logout_user, 
                         login_required)
# login_user, logout_user: creates session in browser, cookie & then deletes
# more secure would be to create the session in the db

@app.route('/register', methods =('GET', 'POST')) # load view and process form
def register():
    form = forms.RegisterForm() #they know to get info from form
    if form.validate_on_submit(): #checks to see if form is posted - and runs validators
        flash("Yay, you registered!", "success") # message category - 2nd one
        models.User.create_user(
            username=form.username.data,
            email=form.email.data,
            password=form.password.data
        )
        return redirect(url_for('index'))
    return render_template('register.html', form=form)
    
@app.route('/')
def index():
    pass

@app.route('/logout') # don't need to add get or post if it is only one
@login_required
def logout():
    logout_user()
    flash("You've been logged out! Come back soon!", "success")
    return redirect(url_for('index'))
    

In [ ]:
# folder templates; register.html
# action="" redirects back here, class is needed bc it is in our CSS
{% from 'macros.html' import render_field %}

<form method="POST" action="" class="form">
    {{ form.hidden_tag() }} #csf safe tags
    {% for field in form %}
    {{ render_field(field)}}
    {% endfor %}
    button type="submit" id="submit">Register!</button>
</form>

# how to not do this above a lot --> create macros.html and import
# into register.html

# macros.html

{% macro render_field(field) %}
    <div class="field">
        {% if field.errors %}
            {% for error infield.errors %}
                <div class="notification error">{{ error }}</div>
            {% endfor %}
        {% endif %}
        {{ field(placeholder=field.label.text) }} #label is the str in Field
    </div>
{% endmacro %}



""" if error about locked db - delete db and restart """

In [ ]:
# With Jinja, when you're dealing with output 
# (stuff to be rendered in the HTML) use {{ }} over {% %}

{% macro hide_email(User) %}
{% set handle, domain = User.email.split("@") %}
{% set hidden = handle[0] + ("*" * handle[1:]|length) %}
{{ hidden + "@" + domain}}
{% endmacro %}

In [ ]:
# login in view and login form

# in forms.py

class LoginForm(Form):
    email = StringField('Email', validators=[DataRequired(), Email()])
    password = PasswordField('Password', validators=[DataRequired()])
    
# in app.py - add import flask_login import login_user
# from flask_bcrypt import check_password_hash

@app.route('/login', methods=('GET', 'POST'))
def login():
    form = forms.LoginForm()
    if form.validate_on_submit():
        try:
            user = models.User.get(models.User.email == form.email.data)
        except models.DoesNotExist:
            flash("Your email or password doesn't match", "error")
        else:
            if check_password_hash(user.password, form.password.data):
                login_user(user)
                flash("You've been logged in!" "success")
                return redirect(url_for('index'))
            else:
                flash("Your email or password doesn't match", "error")
    return render_template('login.html', form=form)

# in login.html - use the same stuff as register :D

{% from 'macros.html' import render_field %}

<form method="POST" action="" class="form">
    {{ form.hidden_tag() }} #csf safe tags
    {% for field in form %}
    {{ render_field(field)}}
    {% endfor %}
    button type="submit" id="submit">Login!</button>
</form>


In [ ]:
# add some layout
